In [13]:
import pandas as pd
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mtick

# Load data:
clark_county_ts2021 = pd.read_csv("H:/My Drive/7. SNV Industry Study/SNV_ArcGIS/IHS Files/Clark_County_TS2021.csv")
bridges_excel = pd.ExcelFile("H:/My Drive/7. SNV Industry Study/SNV_ArcGIS/IHS Files/bridges.xlsx")

# Load necessary sheets from the Excel file
stcc = bridges_excel.parse("STCC")
modes = bridges_excel.parse("Modes")
regions = bridges_excel.parse("Regions")

# Define origin_regions and destination_regions separately
origin_regions = regions.rename(columns={"Region": "Origin Region", "Region Name": "Origin Region Name"})
destination_regions = regions.rename(columns={"Region": "Destination Region", "Region Name": "Destination Region Name"})


# Ensure 'STCC4' column in both dataframes is of type string
# For example, replace NaN with a known string
stcc['STCC4'] = stcc['STCC4'].astype(str)
# naics_stcc['STCC4'] = naics_stcc['STCC4'].astype(str)
# naics_stcc['NAICS_code'] = naics_stcc['NAICS_code'].astype(str)
# Create a new DataFrame from 'stcc' that only includes rows with 2-digit 'STCC' codes
stcc_2digit = stcc[stcc['STCC'].str.len() == 2].copy()
# Ensure 'STCC4' is of type string
stcc_2digit['STCC4'] = stcc_2digit['STCC4'].astype(str)
# Merge stcc_2digit and naics_stcc using STCC4
#stcc_2digit = pd.merge(stcc_2digit, naics_stcc, how='left', on='STCC4')
#stcc = pd.merge(stcc, naics_stcc, how='left', on='STCC4')

# Merge stcc (which now includes naics_stcc data) into clark_county_ts2021 using STCC
clark_county_ts2021 = pd.merge(clark_county_ts2021, stcc, how='left', on='STCC')

# Merge modes into clark_county_ts2021 using Mode
clark_county_ts2021 = pd.merge(clark_county_ts2021, modes, how='left', on='Mode')
# Merge origin_regions and destination_regions into clark_county_ts2021
clark_county_ts2021 = pd.merge(clark_county_ts2021, origin_regions, on='Origin Region', how='left')
clark_county_ts2021 = pd.merge(clark_county_ts2021, destination_regions, on='Destination Region', how='left')

# Create 'STCC_2digit' column in 'clark_county_ts2021' DataFrame
clark_county_ts2021['STCC_2digit'] = clark_county_ts2021['STCC'].str[:2]

# Print DataFrame to check
#print(clark_county_ts2021)
# Rename the rest of the columns
clark_county_ts2021.rename(columns={
    'State_x': 'Origin State', 
    'BEA_x': 'Origin BEA', 
    'BEA Name_x': 'Origin BEA Name', 
    'Country_x': 'Origin Country', 
    'State_y': 'Destination State', 
    'BEA_y': 'Destination BEA', 
    'BEA Name_y': 'Destination BEA Name', 
    'Country_y': 'Destination Country'}, inplace=True)


In [14]:
def filter_data(df):
    outbound = df[(df['Origin Region'] == 32003) & (df['Destination Region'] != 32003)]
    inbound = df[(df['Origin Region'] != 32003) & (df['Destination Region'] == 32003)]
    through = df[(df['Origin Region'] != 32003) & (df['Destination Region'] != 32003)]
    intra = df[(df['Origin Region'] == 32003) & (df['Destination Region'] == 32003)]
    return outbound, inbound, through, intra


In [15]:
def summarize_flow(df, flow_type, geo_column):
    summary = df.groupby(['Year', geo_column]).agg({
        'Value': 'sum',
        'Tons': 'sum'
        # Add other required columns here
    }).reset_index()
    summary.columns = ['Year', 'Region Name', f'{flow_type} Value', f'{flow_type} Tons']
    return summary


In [16]:
# Apply filters to your DataFrame
outbound, inbound, through, intra = filter_data(clark_county_ts2021)

# Summarize values for each flow type using appropriate geographical columns
outbound_summary = summarize_flow(outbound, 'OB', 'Destination Region Name')
inbound_summary = summarize_flow(inbound, 'IB', 'Origin Region Name')
through_origin_summary = summarize_flow(through, 'Thr Origin', 'Origin Region Name')
through_destination_summary = summarize_flow(through, 'Thr Destination', 'Destination Region Name')
intra_summary = summarize_flow(intra, 'Intr', 'Origin Region Name')


In [19]:
# Merge the summaries
final_summary = outbound_summary.merge(inbound_summary, on=['Year', 'Region Name'], how='outer')
final_summary = final_summary.merge(intra_summary, on=['Year', 'Region Name'], how='outer')


In [20]:
# Merge the Through Origin summary with the final summary
final_summary = final_summary.merge(through_origin_summary, left_on=['Year', 'Region Name'], right_on=['Year', 'Region Name'], how='outer')

# Merge the Through Destination summary with the final summary
final_summary = final_summary.merge(through_destination_summary, left_on=['Year', 'Region Name'], right_on=['Year', 'Region Name'], how='outer')


In [24]:
outbound_summary


,Year,Region Name,OB Value,OB Tons
0,2015,"Aberdeen, SD BEA",3.796326e+05,124.626548
1,2015,"Abilene, TX BEA",1.071864e+06,1233.699262
2,2015,"Adams County, ND",3.843786e+02,5.237087
3,2015,Aguascalientes,1.825927e+05,40.342971
4,2015,Alabama Portion of Atlanta BEA,4.654752e+04,17.804393
...,...,...,...,...
1447,2050,Wisconsin Portion of Madison BEA,7.610610e+06,3667.582589
1448,2050,Wisconsin Portion of Minneapolis BEA,1.103530e+06,642.378958
1449,2050,Wyoming Portion of Billings BEA,1.745471e+06,681.506494
1450,2050,Wyoming Portion of Casper BEA,4.986429e+06,3538.599359


In [22]:
outbound_summary

,Year,Region Name,OB Value,OB Tons
0,2015,"Aberdeen, SD BEA",3.796326e+05,124.626548
1,2015,"Abilene, TX BEA",1.071864e+06,1233.699262
2,2015,"Adams County, ND",3.843786e+02,5.237087
3,2015,Aguascalientes,1.825927e+05,40.342971
4,2015,Alabama Portion of Atlanta BEA,4.654752e+04,17.804393
...,...,...,...,...
1447,2050,Wisconsin Portion of Madison BEA,7.610610e+06,3667.582589
1448,2050,Wisconsin Portion of Minneapolis BEA,1.103530e+06,642.378958
1449,2050,Wyoming Portion of Billings BEA,1.745471e+06,681.506494
1450,2050,Wyoming Portion of Casper BEA,4.986429e+06,3538.599359


In [47]:
def summarize_flow(df, flow_type, state_column, country_column):
    group_by_columns = ['Year', state_column, country_column]
    summary = df.groupby(group_by_columns).agg({
        'Value': 'sum',
        'Tons': 'sum'
        # Add other required columns here
    }).reset_index()
    summary.rename(columns={state_column: 'State', country_column: 'Country'}, inplace=True)
    summary.columns = summary.columns.tolist()[:-2] + [f'{flow_type} Value', f'{flow_type} Tons']
    return summary

In [48]:
outbound_summary_ST = summarize_flow(outbound, 'OB', 'Destination State', 'Destination Country')
inbound_summary_ST = summarize_flow(inbound, 'IB', 'Origin State', 'Origin Country')
through_origin_summary_ST = summarize_flow(through, 'Thr Origin', 'Origin State', 'Origin Country')
through_destination_summary_ST = summarize_flow(through, 'Thr Destination', 'Destination State', 'Destination Country')
intra_summary_ST = summarize_flow(intra, 'Intr', 'Origin State', 'Origin Country')


In [49]:
final_summary_ST = outbound_summary_ST.merge(inbound_summary_ST, on=['Year', 'State', 'Country'], how='outer')
final_summary_ST = final_summary_ST.merge(through_origin_summary_ST, on=['Year', 'State', 'Country'], how='outer')
final_summary_ST = final_summary_ST.merge(through_destination_summary_ST, on=['Year', 'State', 'Country'], how='outer')
final_summary_ST = final_summary_ST.merge(intra_summary_ST, on=['Year', 'State', 'Country'], how='outer')


In [50]:
final_summary_ST

,Year,State,Country,OB Value,OB Tons,IB Value,IB Tons,Thr Origin Value,Thr Origin Tons,Thr Destination Value,Thr Destination Tons,Intr Value,Intr Tons
0,2015,AB,CA,8.917196e+07,15384.897980,1.065181e+08,157962.241245,5.302750e+08,529732.438027,8.868482e+08,316563.860516,NaN,NaN
1,2015,AG,MX,1.825927e+05,40.342971,1.152363e+06,133.478185,3.508684e+04,9.018925,1.070827e+05,75.362247,NaN,NaN
2,2015,AK,US,2.478955e+06,1281.417056,5.888916e+07,5253.533728,4.245201e+05,151.407001,NaN,NaN,NaN,NaN
3,2015,AL,US,5.362761e+06,6881.460238,1.506466e+08,48457.066252,1.048254e+07,3012.812838,3.339468e+06,10138.361814,NaN,NaN
4,2015,AR,US,2.818527e+06,1296.663286,1.499748e+08,63698.492881,2.071734e+07,10296.283270,2.266307e+06,4853.660631,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2050,NL,CA,NaN,NaN,NaN,NaN,2.711683e+07,3533.442453,NaN,NaN,NaN,NaN
360,2015,GR,MX,NaN,NaN,NaN,NaN,NaN,NaN,2.226264e+04,107.039710,NaN,NaN
361,2019,GR,MX,NaN,NaN,NaN,NaN,NaN,NaN,1.992906e+04,95.819725,NaN,NaN
362,2021,GR,MX,NaN,NaN,NaN,NaN,NaN,NaN,2.008356e+04,96.562607,NaN,NaN


In [37]:
final_summary

,Year,Region Name,OB Value,OB Tons,IB Value,IB Tons,Intr Value,Intr Tons,Thr Origin Value,Thr Origin Tons,Thr Destination Value,Thr Destination Tons
0,2015,"Aberdeen, SD BEA",3.796326e+05,124.626548,4.798905e+06,2072.707395,NaN,NaN,1.510608e+08,89527.914399,3.742401e+07,13768.420233
1,2015,"Abilene, TX BEA",1.071864e+06,1233.699262,1.765090e+07,12756.702552,NaN,NaN,3.909632e+06,4067.429843,1.216421e+07,186259.195407
2,2015,"Adams County, ND",3.843786e+02,5.237087,1.854684e+04,119.184828,NaN,NaN,1.025102e+06,6316.376702,2.559429e+05,92.196517
3,2015,Aguascalientes,1.825927e+05,40.342971,1.152363e+06,133.478185,NaN,NaN,3.508684e+04,9.018925,1.070827e+05,75.362247
4,2015,Alabama Portion of Atlanta BEA,4.654752e+04,17.804393,1.891225e+06,415.830234,NaN,NaN,NaN,NaN,3.024976e+03,43.326627
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2050,"St. John's, NL CMA",NaN,NaN,NaN,NaN,NaN,NaN,3.270479e+06,340.618836,NaN,NaN
1496,2015,Guerrero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.226264e+04,107.039710
1497,2019,Guerrero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.992906e+04,95.819725
1498,2021,Guerrero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.008356e+04,96.562607


In [51]:
def summarize_flow_by_country(df, flow_type, country_column):
    group_by_columns = ['Year', country_column]
    summary = df.groupby(group_by_columns).agg({
        'Value': 'sum',
        'Tons': 'sum'
        # Add other required columns here
    }).reset_index()
    summary.rename(columns={country_column: 'Country'}, inplace=True)
    summary.columns = summary.columns.tolist()[:-2] + [f'{flow_type} Value', f'{flow_type} Tons']
    return summary


In [52]:
outbound_summary_C = summarize_flow_by_country(outbound, 'OB', 'Destination Country')
inbound_summary_C = summarize_flow_by_country(inbound, 'IB', 'Origin Country')
through_origin_summary_C = summarize_flow_by_country(through, 'Thr Origin', 'Origin Country')
through_destination_summary_C = summarize_flow_by_country(through, 'Thr Destination', 'Destination Country')
intra_summary_C = summarize_flow_by_country(intra, 'Intr', 'Origin Country')


In [53]:
final_summary_C = outbound_summary_C.merge(inbound_summary_C, on=['Year', 'Country'], how='outer')
final_summary_C = final_summary_C.merge(through_origin_summary_C, on=['Year', 'Country'], how='outer')
final_summary_C = final_summary_C.merge(through_destination_summary_C, on=['Year', 'Country'], how='outer')
final_summary_C = final_summary_C.merge(intra_summary_C, on=['Year', 'Country'], how='outer')


In [54]:
final_summary_C

,Year,Country,OB Value,OB Tons,IB Value,IB Tons,Thr Origin Value,Thr Origin Tons,Thr Destination Value,Thr Destination Tons,Intr Value,Intr Tons
0,2015,CA,2.740712e+08,6.972143e+04,3.789786e+08,2.585833e+05,1.422369e+10,2.355831e+06,5.879073e+09,1.284952e+06,NaN,NaN
1,2015,MX,5.694821e+08,9.782909e+04,4.674276e+08,1.182158e+05,1.410144e+09,1.527795e+05,1.716161e+09,8.421733e+05,NaN,NaN
2,2015,US,5.359368e+09,6.016162e+06,2.737902e+10,1.724997e+07,1.060750e+11,4.819453e+07,1.141137e+11,4.857602e+07,8.165316e+09,1.610371e+07
3,2019,CA,2.782379e+08,6.178344e+04,4.950276e+08,2.856832e+05,1.304373e+10,2.124855e+06,5.325549e+09,1.287333e+06,NaN,NaN
4,2019,MX,3.517738e+08,6.529158e+04,6.202555e+08,1.546833e+05,1.342453e+09,1.590279e+05,6.947760e+08,4.260010e+05,NaN,NaN
5,2019,US,6.044934e+09,5.811412e+06,2.715480e+10,1.812372e+07,9.953154e+10,4.633434e+07,1.078974e+11,4.690489e+07,9.045947e+09,1.633485e+07
6,2021,CA,3.456530e+08,6.752876e+04,7.221300e+08,4.163034e+05,1.231587e+10,2.519731e+06,6.196029e+09,1.423040e+06,NaN,NaN
7,2021,MX,3.901267e+08,7.220680e+04,7.931717e+08,2.070051e+05,1.518588e+09,1.831356e+05,7.006417e+08,4.713985e+05,NaN,NaN
8,2021,US,6.238153e+09,5.865877e+06,2.764964e+10,1.784366e+07,1.005795e+11,4.593842e+07,1.075173e+11,4.674685e+07,9.073076e+09,1.606648e+07
9,2050,CA,5.936028e+08,1.395114e+05,1.519939e+09,8.671547e+05,2.166383e+10,5.499458e+06,1.223031e+10,2.702818e+06,NaN,NaN


In [55]:
through_destination_summary_C

,Year,Country,Thr Destination Value,Thr Destination Tons
0,2015,CA,5.879073e+09,1.284952e+06
1,2015,MX,1.716161e+09,8.421733e+05
2,2015,US,1.141137e+11,4.857602e+07
3,2019,CA,5.325549e+09,1.287333e+06
4,2019,MX,6.947760e+08,4.260010e+05
5,2019,US,1.078974e+11,4.690489e+07
6,2021,CA,6.196029e+09,1.423040e+06
7,2021,MX,7.006417e+08,4.713985e+05
8,2021,US,1.075173e+11,4.674685e+07
9,2050,CA,1.223031e+10,2.702818e+06


In [56]:
through_origin_summary_C

,Year,Country,Thr Origin Value,Thr Origin Tons
0,2015,CA,1.422369e+10,2.355831e+06
1,2015,MX,1.410144e+09,1.527795e+05
2,2015,US,1.060750e+11,4.819453e+07
3,2019,CA,1.304373e+10,2.124855e+06
4,2019,MX,1.342453e+09,1.590279e+05
5,2019,US,9.953154e+10,4.633434e+07
6,2021,CA,1.231587e+10,2.519731e+06
7,2021,MX,1.518588e+09,1.831356e+05
8,2021,US,1.005795e+11,4.593842e+07
9,2050,CA,2.166383e+10,5.499458e+06


In [61]:
total_sum = through_origin_summary_C['Thr Origin Value'].sum()
print(f"The total sum of the through_origin_summary_C 'Thr Origin Value' is {total_sum}")
total_sum = through_destination_summary_C['Thr Destination Value'].sum()
print(f"The total sum of the through_destination_summary_C 'Thr Destination Value' is {total_sum}")

total_sum = through_origin_summary['Thr Origin Value'].sum()
print(f"The total sum of the through_origin_summary 'Thr Origin Value' is {total_sum}")

total_sum = through_destination_summary['Thr Destination Value'].sum()
print(f"The total sum of the through_destination_summary 'Thr Destination Value' is {total_sum}")


The total sum of the through_origin_summary_C 'Thr Origin Value' is 552841471775.4816
The total sum of the through_destination_summary_C 'Thr Destination Value' is 552841471775.4811
The total sum of the through_origin_summary 'Thr Origin Value' is 552841471775.4436
The total sum of the through_destination_summary 'Thr Destination Value' is 552841471775.4507


In [59]:
through_origin_summary


,Year,Region Name,Thr Origin Value,Thr Origin Tons
0,2015,"Aberdeen, SD BEA",1.510608e+08,8.952791e+04
1,2015,"Abilene, TX BEA",3.909632e+06,4.067430e+03
2,2015,"Adams County, ND",1.025102e+06,6.316377e+03
3,2015,Aguascalientes,3.508684e+04,9.018925e+00
4,2015,Alabama Portion of Columbus BEA,2.092286e+05,2.299999e+02
...,...,...,...,...
1235,2050,Wisconsin Portion of Madison BEA,2.083883e+09,7.405164e+05
1236,2050,Wisconsin Portion of Minneapolis BEA,5.944888e+08,2.415466e+05
1237,2050,Wyoming Portion of Billings BEA,1.453712e+08,1.700689e+05
1238,2050,Wyoming Portion of Casper BEA,8.226121e+08,1.279076e+06
